In [ ]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, TFBertModel
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import torch

In [ ]:
def load_data(data_file):
  # read csv file
  df = pd.read_csv(data_file)
  df.fillna("", inplace=True)
  comments = df['comment'].tolist()
  genders = df['user_gender'].tolist()
  genders = [0 if gender == "Male" else 1 for gender in genders]

  return comments, genders

In [ ]:
# Load dataset
comments, genders = load_data('C:/Users/ADMIN PC/Desktop/Comment/Comment_dataset.csv')

In [ ]:
comments = np.array(comments)
genders = np.array(genders)

In [ ]:
MAX_SEQUENCE_LENGTH = 300
MAX_NUM_WORDS = 50000

In [ ]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)

In [ ]:
tokenizer.fit_on_texts(comments)
sequences = tokenizer.texts_to_sequences(comments)

In [ ]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(genders)

In [ ]:
data.shape, labels.shape

((190104, 300), (190104, 2))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels,
                                                    test_size=0.2,
                                                    stratify=genders,
                                                    random_state=42)

In [ ]:
EMBEDDING_DIM = 768
num_words = MAX_NUM_WORDS
embedding_layer = Embedding(num_words,EMBEDDING_DIM,input_length=MAX_SEQUENCE_LENGTH,trainable=True)

In [ ]:
sequence_input = Input(shape=(300, ))
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(64, kernel_size=3, activation='relu')(embedded_sequences)
x = Conv1D(64, kernel_size=3, activation='relu')(x)
x = MaxPooling1D(pool_size=2)(x)
x = Flatten()(x)
x = Dense(100, activation='relu')(x)
preds = Dense(2, activation='sigmoid')(x)

model = Model(sequence_input, preds)

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['acc'])

In [ ]:
model.summary()

In [ ]:
model.fit(X_train, y_train,batch_size=50, epochs=10, validation_data=(X_test, y_test))

In [ ]:
model.save("C:/Users/ADMIN PC/Desktop/Comment/CNN/CNN.h5")

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

compile_metrics: 77.26%


In [ ]:
def predict_proba(arr):
  sequences_new = tokenizer.texts_to_sequences(arr)
  data = pad_sequences(sequences_new, maxlen=MAX_SEQUENCE_LENGTH)
  yprob = model.predict(data, verbose=None)
  # yclasses=yprob.argmax(axis=-1)

  returnable=[]
  for i in yprob:
    temp=i[0]
    returnable.append(np.array([temp,1- temp]))
  return np.array(returnable)

In [ ]:
def predict_male_or_female(txt):
    arr = np.array([txt])
    sequences_new = tokenizer.texts_to_sequences(arr)
    data = pad_sequences(sequences_new, maxlen=MAX_SEQUENCE_LENGTH)
    yprob = model.predict(data)[0]
    gender = "Female" if yprob[1] >= 0.5 else "Male"

    return gender

In [ ]:
txt = "I've had the 50watter since Oct of last year and I'm still impressed and blown away every time I play it. It's unreal. Killer choice"
predict_male_or_female(txt)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step


'Male'

In [ ]:
txt = "You l've got a good man there hun. Take care of each other and it'll last a long time.\nFor sore throats my dad used to take 2 tablespoons of apple cider vinegar and the same amount of honey, mix it in at least 8oz of hot water. Drink it while ot's still hot, but not burning. Worked every time for me...still does."
predict_male_or_female(txt)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


'Female'